# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:

import helper

data_dir = './data/simpsons/moes_tavern_lines.txt' 

text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
view_sentence_range = (0, 10)


import numpy as np

print('Dataset Stats')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

Dataset Stats
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 15.248091603053435
Number of lines: 4257
Average number of words in each line: 11.50434578341555

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [3]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    
    from collections import Counter 
    
    counts = Counter(text) 
    vocab = sorted(counts, key=counts.get, reverse=True) 
    vocab_to_int = {word: ii for ii, word in enumerate(vocab)}  
    print(vocab_to_int)
    int_to_vocab = {v:k for k, v in vocab_to_int.items()}  
    print(int_to_vocab) 
    
    return (vocab_to_int, int_to_vocab)


tests.test_create_lookup_tables(create_lookup_tables)

{'hey': 7, "moe's": 20, 'seen': 22, "i'll": 23, 'tavern': 45, 'back': 25, 'gonna': 8, 'should': 17, 'i': 26, 'elite': 27, 'got': 28, 'eh': 64, 'days': 30, 'homer_simpson': 31, 'has': 32, 'little': 33, 'listen': 34, 'social': 35, 'effervescent': 57, 'one': 14, 'forget': 38, 'enhance': 39, 'and': 40, 'self': 41, 'name': 9, 'an': 37, 'not': 11, 'catch': 42, 'on': 12, 'mike': 1, 'where': 43, 'there': 47, 'another': 44, 'hello': 46, 'you': 2, 'homer': 10, 'drink': 5, 'my': 13, 'skills': 48, 'pick': 49, 'to': 6, 'only': 50, 'carve': 51, 'matter': 52, 'rotch': 3, 'give': 21, 'bart_simpson': 53, 'me': 54, "i'm": 15, 'whats': 55, "you're": 56, 'problems': 16, 'of': 58, 'puke': 36, 'last': 59, 'normal': 60, 'these': 66, 'hold': 61, 'is': 62, 'yeah': 18, 'the': 19, 'check': 63, 'with': 29, 'anybody': 65, 'ice': 24, 'moe_szyslak': 0, 'moe': 67, 'barney_gumble': 68, 'meet': 69, 'your': 4, 'lately': 70}
{0: 'moe_szyslak', 1: 'mike', 2: 'you', 3: 'rotch', 4: 'your', 5: 'drink', 6: 'to', 7: 'hey', 8: 

### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

In [4]:
def token_lookup():
   
    d = {
        '.': '||period||',
        ',': '||comma||',
        '"': '||quotation_mark||',
        ';': '||semicolon||',
        '!': '||exclamation_mark||',
        '?': '||question_mark||',
        '(': '||left_parentheses||',
        ')': '||right_parentheses||',
        '--': '||dash||',
        '\n': '||return||',
    }
    return d
    

tests.test_tokenize(token_lookup)

Tests Passed


## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [5]:

# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

{'dames': 1903, 'left': 423, 'rhode': 2899, 'wagering': 2900, 'microwave': 2901, 'system': 1441, 'blooded': 2902, 'dearest': 1904, 'yea': 961, 'beatings': 2903, 'seymour': 513, 'specified': 2904, 'sincerely': 2905, 'suit': 963, 'stuck': 1905, 'marvelous': 2907, 'cries': 1170, 'faint': 2908, 'terror': 2909, 'ziff': 1906, 'girlfriend': 1171, 'rome': 2910, 'stones': 2911, 'rainforest': 2912, 'obama': 2914, 'bought': 1442, 'expense': 2915, 'person': 604, 'wang': 1907, 'beached': 2916, 'stands': 1908, 'moonshine': 5523, 'devastated': 1909, 'flash-fry': 4878, 'sips': 841, 'size': 1444, 'apu_nahasapeemapetilon:': 241, 'drains': 6135, 'open': 664, 'forward': 1910, 'nightmare': 2918, 'outside': 549, 'attention': 1445, 'fountain': 2919, 'gift': 1446, 'whim': 2920, 'check': 267, 'being': 398, 'body': 1172, 'champion': 2921, 'flaming': 471, 'divorced': 2922, 'football': 2923, 'cerebral': 6331, 'chuck': 4200, 'laney_fontaine:': 1944, 'relaxing': 5525, 'sitting': 730, 'cheese': 1911, 'y-you': 2927, 

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [6]:
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [7]:
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.0.0
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following the tuple `(Input, Targets, LearingRate)`

In [8]:
def get_inputs():
    
    input_ = tf.placeholder(tf.int32, [None, None], name = "input")
    targets = tf.placeholder(tf.int32, [None, None]) 
    learning_rate = tf.placeholder(tf.float32)
    
    return (input_, targets, learning_rate)

tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

In [9]:
def get_init_cell(batch_size, rnn_size):
   
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size) 
    drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob = 0.5)
    cell = tf.contrib.rnn.MultiRNNCell([drop])  
    initialize_state = tf.identity(cell.zero_state(batch_size, dtype = tf.float32), name = "initial_state") 
    return (cell, initialize_state)


tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [10]:
def get_embed(input_data, vocab_size, embed_dim):
    
    embedd_weights = tf.Variable(tf.random_uniform([vocab_size, embed_dim], -1.0, 1.0)) 
    embed = tf.nn.embedding_lookup(embedd_weights, input_data)
    return embed

tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [11]:
def build_rnn(cell, inputs):
    
    outputs,final_state = tf.nn.dynamic_rnn(cell,inputs,dtype = tf.float32)
    final_state = tf.identity(final_state, name = "final_state")    
    return (outputs, final_state)

tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [12]:
def build_nn(cell, rnn_size, input_data, vocab_size):
    
    embedd = get_embed(input_data, vocab_size, rnn_size) 
    outputs, finalState = build_rnn(cell, embedd) 
    logits = tf.contrib.layers.fully_connected(outputs, vocab_size, activation_fn = None) 
    return (logits,finalState)
    

tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [13]:


def get_batches(int_text, batch_size, seq_length):
    
    n_batches = int(len(int_text) / (batch_size * seq_length))

    xdata = np.array(int_text[: n_batches * batch_size * seq_length])
    ydata = np.array(int_text[1: n_batches * batch_size * seq_length + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)
    
    return np.asarray(list(zip(x_batches, y_batches)))
    


tests.test_get_batches(get_batches)

Tests Passed


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [17]:
# Number of Epochs
num_epochs = 90
# Batch Size
batch_size = 750
# RNN Size
rnn_size = 1024
# Sequence Length
seq_length = 5
# Learning Rate
learning_rate = 0.005
# Show stats for every n number of batches
show_every_n_batches = 10


save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [18]:
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [19]:
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/18   train_loss = 8.822
Epoch   0 Batch   10/18   train_loss = 5.717
Epoch   1 Batch    2/18   train_loss = 5.015
Epoch   1 Batch   12/18   train_loss = 4.800
Epoch   2 Batch    4/18   train_loss = 4.327
Epoch   2 Batch   14/18   train_loss = 4.115
Epoch   3 Batch    6/18   train_loss = 3.831
Epoch   3 Batch   16/18   train_loss = 3.599
Epoch   4 Batch    8/18   train_loss = 3.272
Epoch   5 Batch    0/18   train_loss = 2.984
Epoch   5 Batch   10/18   train_loss = 2.802
Epoch   6 Batch    2/18   train_loss = 2.683
Epoch   6 Batch   12/18   train_loss = 2.556
Epoch   7 Batch    4/18   train_loss = 2.343
Epoch   7 Batch   14/18   train_loss = 2.317
Epoch   8 Batch    6/18   train_loss = 2.105
Epoch   8 Batch   16/18   train_loss = 2.043
Epoch   9 Batch    8/18   train_loss = 1.919
Epoch  10 Batch    0/18   train_loss = 1.824
Epoch  10 Batch   10/18   train_loss = 1.775
Epoch  11 Batch    2/18   train_loss = 1.722
Epoch  11 Batch   12/18   train_loss = 1.678
Epoch  12 

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [20]:
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [21]:
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [27]:
def get_tensors(loaded_graph):
    InputTensor = loaded_graph.get_tensor_by_name("input:0") 
    InitialStateTensor = loaded_graph.get_tensor_by_name("initial_state:0")
    FinalStateTensor = loaded_graph.get_tensor_by_name("final_state:0")
    ProbsTensor = loaded_graph.get_tensor_by_name("probs:0")
    return (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)

tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [24]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    int_word = np.argmax(probabilities)
    return int_to_vocab[int_word]

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [25]:
gen_length = 200
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        pred_word = pick_word(probabilities[dyn_seq_length-1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

moe_szyslak:(accusing) mrs. _powers: i could do is be surprised.
professor_jonathan_frink: who wants to party, right, and i can toss back a at my old.(desperate) there's gotta make it a grand.
moe_szyslak: oh, no... inclination...
moe_szyslak:(shocked) are you nuts? you got that in?
homer_simpson:(explaining)" to go to the--(then) there's a first place, homer. i never seen a lotta talkin' about a girl school his wife is for the guys in the morning. i promise.
moe_szyslak: hey marge, get back.
moe_szyslak:(to little) moe, moe, you sure look sad.
moe_szyslak:(into phone) hello...
all:(singing) hello,....
moe_szyslak: plus he fixed the cheap...(points to door through himself)
homer_simpson: that was an eyeball.
lisa_simpson: dad, i am.
carl_carlson: all right, but i am not cursed!
bart_simpson:


# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.